# 欄名稱轉成變量值

- [Pandas 与数据整理 | 张吉的博客](http://shzhangji.com/cnblogs/2017/09/30/pandas-and-tidy-data/)

## 列名称是数据值，而非变量名

### 宗教信仰与收入 - Pew 论坛

#### 法1:使用stack

In [1]:
import pandas as pd
df = pd.read_csv('data/pew.csv')
df.head(10)

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Don’t know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovah's Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


In [2]:
df = df.set_index('religion')
df = df.stack()
df.index = df.index.rename('income', level=1)
df.name = 'frequency'
df = df.reset_index()
df.head(10)

,religion,income,frequency
0,Agnostic,<$10k,27
1,Agnostic,$10-20k,34
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Atheist,<$10k,12
7,Atheist,$10-20k,27
8,Atheist,$20-30k,37
9,Atheist,$30-40k,52


#### 法2:使用melt

In [4]:
df = pd.read_csv('data/pew.csv')
df = pd.melt(df, id_vars=['religion'], value_vars=list(df.columns)[1:],
             var_name='income', value_name='frequency')
df = df.sort_values(by='religion')
df.to_csv('data/pew-tidy.csv', index=False)
df.head(10)

,religion,income,frequency
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34
20,Agnostic,$20-30k,60
41,Atheist,$40-50k,35
21,Atheist,$20-30k,37
11,Atheist,$10-20k,27
31,Atheist,$30-40k,52


### Billboard 2000

In [26]:
df = pd.read_csv('data/billboard.csv')
df

,year,artist,track,time,date.entered,wk1,wk2,wk3
0,2000,2 Pac,Baby Don't Cry,4:22,2000-02-26,87,82,72
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87,92
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70,68
3,2000,98^0,Give Me Just One Nig...,3:24,2000-08-19,51,39,34
4,2000,A*Teens,Dancing Queen,3:44,2000-07-08,97,97,96
5,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,84,62,51
6,2000,Aaliyah,Try Again,4:03,2000-03-18,59,53,38
7,2000,"Adams, Yolanda",Open My Heart,5:30,2000-08-26,76,76,74


In [27]:
df = pd.melt(df, id_vars=list(df.columns)[:5], value_vars=list(df.columns)[5:],
             var_name='week', value_name='rank')

df['week'] = df['week'].str[2:].astype(int)
df['date.entered'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta((df['week'] - 1) * 7, 'd')
df = df.rename(columns={'date.entered': 'date'})
df = df.sort_values(by=['track', 'date'])
df.to_csv('data/billboard-intermediate.csv', index=False)
df.head(10)

,year,artist,track,time,date,week,rank
0,2000,2 Pac,Baby Don't Cry,4:22,2000-02-26,1,87
8,2000,2 Pac,Baby Don't Cry,4:22,2000-03-04,2,82
16,2000,2 Pac,Baby Don't Cry,4:22,2000-03-11,3,72
4,2000,A*Teens,Dancing Queen,3:44,2000-07-08,1,97
12,2000,A*Teens,Dancing Queen,3:44,2000-07-15,2,97
20,2000,A*Teens,Dancing Queen,3:44,2000-07-22,3,96
3,2000,98^0,Give Me Just One Nig...,3:24,2000-08-19,1,51
11,2000,98^0,Give Me Just One Nig...,3:24,2000-08-26,2,39
19,2000,98^0,Give Me Just One Nig...,3:24,2000-09-02,3,34
5,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,1,84


## 一列包含多个变量

### 结核病 (TB)

In [60]:
df = pd.read_csv('data/tb.csv')
df

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0,0,1,0,0,0,0,---,---
1,AE,2000,2,4,4,6,5,12,10,---,3
2,AF,2000,52,228,183,149,129,94,80,---,93
3,AG,2000,0,0,0,0,0,0,1,---,1
4,AL,2000,2,19,21,14,24,19,16,---,3
5,AM,2000,2,152,130,131,63,26,21,---,1
6,AN,2000,0,0,1,2,0,0,0,---,0
7,AO,2000,186,999,1003,912,482,312,194,---,247
8,AR,2000,97,278,594,402,419,368,330,---,121
9,AS,2000,---,---,---,---,1,1,---,---,---


In [8]:
df = pd.melt(df, id_vars=['country', 'year'], value_vars=list(df.columns)[2:],
             var_name='column', value_name='cases')
df = df[df['cases'] != '---']
df['cases'] = df['cases'].astype(int)
df['sex'] = df['column'].str[0]
df['age'] = df['column'].str[1:].map({
    '014': '0-14',
    '1524': '15-24',
    '2534': '25-34',
    '3544': '35-44',
    '4554': '45-54',
    '5564': '55-64',
    '65': '65+'
})
df = df[['country', 'year', 'sex', 'age', 'cases']]
df.to_csv('data/tb-tidy.csv', index=False)
df.head(10)

,country,year,sex,age,cases
0,AD,2000,m,0-14,0
1,AE,2000,m,0-14,2
2,AF,2000,m,0-14,52
3,AG,2000,m,0-14,0
4,AL,2000,m,0-14,2
5,AM,2000,m,0-14,2
6,AN,2000,m,0-14,0
7,AO,2000,m,0-14,186
8,AR,2000,m,0-14,97
10,AD,2000,m,15-24,0


### set_index() 與stack() 的用法

In [61]:
df2 = df.set_index(['country', 'year'])

In [64]:
df2.head(10)

,,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
country,year,,,,,,,,,
AD,2000,0,0,1,0,0,0,0,---,---
AE,2000,2,4,4,6,5,12,10,---,3
AF,2000,52,228,183,149,129,94,80,---,93
AG,2000,0,0,0,0,0,0,1,---,1
AL,2000,2,19,21,14,24,19,16,---,3
AM,2000,2,152,130,131,63,26,21,---,1
AN,2000,0,0,1,2,0,0,0,---,0
AO,2000,186,999,1003,912,482,312,194,---,247
AR,2000,97,278,594,402,419,368,330,---,121


In [65]:
df2 = df2.stack()

In [66]:
pd.DataFrame(df2).head(10)

0
country year           
AD      2000 m014     0
             m1524    0
             m2534    1
             m3544    0
             m4554    0
             m5564    0
             m65      0
             mu     ---
             f014   ---
AE      2000 m014     2

In [67]:
df2.index = df2.index.rename('label', level=2)

In [69]:
pd.DataFrame(df2).head(10)

0
country year label     
AD      2000 m014     0
             m1524    0
             m2534    1
             m3544    0
             m4554    0
             m5564    0
             m65      0
             mu     ---
             f014   ---
AE      2000 m014     2

In [70]:
df2.name = 'text'

In [71]:
pd.DataFrame(df2).head(10)

text
country year label     
AD      2000 m014     0
             m1524    0
             m2534    1
             m3544    0
             m4554    0
             m5564    0
             m65      0
             mu     ---
             f014   ---
AE      2000 m014     2

In [72]:
df2 = df2.reset_index()

In [74]:
df2.head(10)

,country,year,label,text
0,AD,2000,m014,0
1,AD,2000,m1524,0
2,AD,2000,m2534,1
3,AD,2000,m3544,0
4,AD,2000,m4554,0
5,AD,2000,m5564,0
6,AD,2000,m65,0
7,AD,2000,mu,---
8,AD,2000,f014,---
9,AE,2000,m014,2


## 变量存储在行和列中

### 气象站

In [28]:
df = pd.read_csv('data/weather.csv')
df

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,---,---,---,---,---,---,---,---
1,MX17004,2010,1,tmin,---,---,---,---,---,---,---,---
2,MX17004,2010,2,tmax,---,27.3,24.1,---,---,---,---,---
3,MX17004,2010,2,tmin,---,14.4,14.4,---,---,---,---,---
4,MX17004,2010,3,tmax,---,---,---,---,32.1,---,---,---
5,MX17004,2010,3,tmin,---,---,---,---,14.2,---,---,---
6,MX17004,2010,4,tmax,---,---,---,---,---,---,---,---
7,MX17004,2010,4,tmin,---,---,---,---,---,---,---,---
8,MX17004,2010,5,tmax,---,---,---,---,---,---,---,---
9,MX17004,2010,5,tmin,---,---,---,---,---,---,---,---


In [29]:

df = pd.melt(df, id_vars=['id', 'year', 'month', 'element'],
             value_vars=list(df.columns)[4:],
             var_name='date', value_name='value')
df['date'] = df['date'].str[1:].astype('int')
df['date'] = df[['year', 'month', 'date']].apply(
    lambda row: '{:4d}-{:02d}-{:02d}'.format(*row),
    axis=1)
df = df.loc[df['value'] != '---', ['id', 'date', 'element', 'value']]
df = df.set_index(['id', 'date', 'element'])
df = df.unstack()
df.columns = list(df.columns.get_level_values('element'))
df = df.reset_index()
df.to_csv('data/weather-tidy.csv', index=False)
df

,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


### 同一表中包含多种观测类型

### Billboard 2000

In [30]:
df = pd.read_csv('data/billboard-intermediate.csv')
df

,year,artist,track,time,date,week,rank
0,2000,2 Pac,Baby Don't Cry,4:22,2000-02-26,1,87
1,2000,2 Pac,Baby Don't Cry,4:22,2000-03-04,2,82
2,2000,2 Pac,Baby Don't Cry,4:22,2000-03-11,3,72
3,2000,A*Teens,Dancing Queen,3:44,2000-07-08,1,97
4,2000,A*Teens,Dancing Queen,3:44,2000-07-15,2,97
5,2000,A*Teens,Dancing Queen,3:44,2000-07-22,3,96
6,2000,98^0,Give Me Just One Nig...,3:24,2000-08-19,1,51
7,2000,98^0,Give Me Just One Nig...,3:24,2000-08-26,2,39
8,2000,98^0,Give Me Just One Nig...,3:24,2000-09-02,3,34
9,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,1,84


In [31]:

df_track = df[['artist', 'track', 'time']].drop_duplicates()
df_track.insert(0, 'id', range(1, len(df_track) + 1))
df = pd.merge(df, df_track, on=['artist', 'track', 'time'])
df = df[['id', 'date', 'rank']]
df_track.to_csv('data/billboard-track.csv', index=False)
df.to_csv('data/billboard-rank.csv', index=False)
print(df_track, '\n\n', df)

    id          artist                    track  time
0    1           2 Pac           Baby Don't Cry  4:22
3    2         A*Teens            Dancing Queen  3:44
6    3            98^0  Give Me Just One Nig...  3:24
9    4         Aaliyah            I Don't Wanna  4:15
12   5    3 Doors Down               Kryptonite  3:53
15   6  Adams, Yolanda            Open My Heart  5:30
18   7         2Ge+her  The Hardest Part Of ...  3:15
21   8         Aaliyah                Try Again  4:03 

     id        date  rank
0    1  2000-02-26    87
1    1  2000-03-04    82
2    1  2000-03-11    72
3    2  2000-07-08    97
4    2  2000-07-15    97
5    2  2000-07-22    96
6    3  2000-08-19    51
7    3  2000-08-26    39
8    3  2000-09-02    34
9    4  2000-01-29    84
10   4  2000-02-05    62
11   4  2000-02-12    51
12   5  2000-04-08    81
13   5  2000-04-15    70
14   5  2000-04-22    68
15   6  2000-08-26    76
16   6  2000-09-02    76
17   6  2000-09-09    74
18   7  2000-09-02    91
19   7  200

### wide_to_long() 範例

In [32]:
df = pd.DataFrame({
    'famid': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'birth': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'ht1': [2.8, 2.9, 2.2, 2, 1.8, 1.9, 2.2, 2.3, 2.1],
    'ht2': [3.4, 3.8, 2.9, 3.2, 2.8, 2.4, 3.3, 3.4, 2.9]
})

In [33]:
df

,famid,birth,ht1,ht2
0,1,1,2.8,3.4
1,1,2,2.9,3.8
2,1,3,2.2,2.9
3,2,1,2.0,3.2
4,2,2,1.8,2.8
5,2,3,1.9,2.4
6,3,1,2.2,3.3
7,3,2,2.3,3.4
8,3,3,2.1,2.9


In [34]:
l = pd.wide_to_long(df, stubnames='ht', i=['famid', 'birth'], j='age')

In [35]:
l

ht
famid birth age     
1     1     1    2.8
            2    3.4
      2     1    2.9
            2    3.8
      3     1    2.2
            2    2.9
2     1     1    2.0
            2    3.2
      2     1    1.8
            2    2.8
      3     1    1.9
            2    2.4
3     1     1    2.2
            2    3.3
      2     1    2.3
            2    3.4
      3     1    2.1
            2    2.9

In [54]:
w = l.unstack()
w

ht     
age            1    2
famid birth          
1     1      2.8  3.4
      2      2.9  3.8
      3      2.2  2.9
2     1      2.0  3.2
      2      1.8  2.8
      3      1.9  2.4
3     1      2.2  3.3
      2      2.3  3.4
      3      2.1  2.9

In [55]:
w.columns

MultiIndex(levels=[['ht'], [1, 2]],
           codes=[[0, 0], [0, 1]],
           names=[None, 'age'])

In [56]:
w.columns.map(print)

('ht', 1)
('ht', 2)


Index([None, None], dtype='object')

In [58]:
# w.columns = w.columns.map('{0[0]}{0[1]}'.format)
w.columns = w.columns.map(lambda x: f'{x[0]}{x[1]}')
w.columns

Index(['ht', 'ht'], dtype='object')

In [59]:
w.reset_index()

,famid,birth,ht,ht
0,1,1,2.8,3.4
1,1,2,2.9,3.8
2,1,3,2.2,2.9
3,2,1,2.0,3.2
4,2,2,1.8,2.8
5,2,3,1.9,2.4
6,3,1,2.2,3.3
7,3,2,2.3,3.4
8,3,3,2.1,2.9
